# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [2]:
# try this project here!

In [3]:
import syft as sy
import torch as th

hook = sy.TorchHook(th)
from torch import nn, optim

In [4]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secured_worker = sy.VirtualWorker(hook, id="secured_worker")

In [5]:
bob.add_workers([alice, secured_worker])
alice.add_workers([bob, secured_worker])
secured_worker.add_workers([bob, alice])

<VirtualWorker id:secured_worker #objects:0>

In [6]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [7]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:4].send(alice)
alices_target = target[2:4].send(alice)

In [8]:
# A Toy Model
model = nn.Linear(2,1)

In [9]:
for num_rounds in range(10):
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)
    
    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
    
    for i in range(10):
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) **2).sum()
        bobs_loss.backward()
        
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data
        bobs_loss
        
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) **2).sum()
        alices_loss.backward()
        
        alices_opt.step()
        alices_loss = alices_loss.get().data
        alices_loss
        
    alices_model.move(secured_worker)
    bobs_model.move(secured_worker)
    
    with th.no_grad():
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bobs loss: " + str(bobs_loss) + "Alices loss: " + str(alices_loss))

Bobs loss: tensor(0.0786)Alices loss: tensor(0.0486)
Bobs loss: tensor(0.0242)Alices loss: tensor(0.0053)
Bobs loss: tensor(0.0068)Alices loss: tensor(2.7703e-05)
Bobs loss: tensor(0.0022)Alices loss: tensor(0.0005)
Bobs loss: tensor(0.0009)Alices loss: tensor(0.0009)
Bobs loss: tensor(0.0004)Alices loss: tensor(0.0010)
Bobs loss: tensor(0.0002)Alices loss: tensor(0.0009)
Bobs loss: tensor(0.0002)Alices loss: tensor(0.0007)
Bobs loss: tensor(0.0001)Alices loss: tensor(0.0006)
Bobs loss: tensor(7.8760e-05)Alices loss: tensor(0.0004)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [10]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [11]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [12]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [13]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [14]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [15]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [16]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [17]:
# try this project here!

In [18]:
import random
Q = 23740629843760239486723

In [19]:
def encrypt(number, num_shares=3):
    shares = list()
    
    for i in range(num_shares - 1):
        shares.append(random.randint(0, Q))
    
    last_share = Q - (sum(shares) % Q) + number
    shares.append(last_share)
    
    return tuple(shares)

In [20]:
def decrypt(shares):
    return sum(shares) % Q

In [21]:
def add(user1, user2):
    new_user = list()
    
    assert(len(user1) == len(user2))
    
    for i in range(len(user1)):
        new_user.append((user1[i] + user2[i]) % Q)
        
    return tuple(new_user)

In [22]:
encrypt(7)

(10185070099810157533463, 1152273775750654235919, 12403285968199427717348)

In [23]:
decrypt(encrypt(5, 6))

5

In [24]:
decrypt(add(encrypt(5, 3), encrypt(10, 3)))

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [25]:
BASE=10
PRECISION=4

In [26]:
def encode(x):
    return int(x * (BASE ** PRECISION)) % Q

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [27]:
encode(-0.50)

23740629843760239481723

In [28]:
decode(23740629843760239481723)

-0.5

In [29]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [63]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secured_worker = secured_worker.clear_objects()

In [64]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [57]:
x = x.share(bob, alice, secured_worker)
x

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:86115903831 -> bob:37230448560]
	-> (Wrapper)>[PointerTensor | me:42557289339 -> alice:52068370226]
	-> (Wrapper)>[PointerTensor | me:81761874213 -> secured_worker:44815539042]
	*crypto provider: me*

In [58]:
bob._objects

{37230448560: tensor([1065752611398878919, 2780805867124373579, 3103306277749802174,
         1246454115997732055, 2109936899542952472])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [59]:
y = x + x

In [60]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:50303084233 -> bob:45454278500]
	-> (Wrapper)>[PointerTensor | me:86704836190 -> alice:47376858104]
	-> (Wrapper)>[PointerTensor | me:26465614405 -> secured_worker:87272315940]
	*crypto provider: me*

In [36]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [37]:
x = th.tensor([0.1,0.2,0.3])

In [38]:
x

tensor([0.1000, 0.2000, 0.3000])

In [39]:
x = x.fix_prec()
x

(Wrapper)>FixedPrecisionTensor>tensor([100, 200, 300])

In [40]:
x.child.child

tensor([100, 200, 300])

In [41]:
y = x + x

In [42]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [43]:
x = th.tensor([0.1, 0.2, 0.3])

In [44]:
x = x.fix_prec().share(bob, alice, secured_worker)
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:83702799569 -> bob:28862231340]
	-> (Wrapper)>[PointerTensor | me:58538925511 -> alice:76444179276]
	-> (Wrapper)>[PointerTensor | me:42514964139 -> secured_worker:55727768704]
	*crypto provider: me*

In [45]:
y = x + x
y

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:36506568887 -> bob:25235630671]
	-> (Wrapper)>[PointerTensor | me:36716845056 -> alice:52038003767]
	-> (Wrapper)>[PointerTensor | me:65257850841 -> secured_worker:85838750289]
	*crypto provider: me*

In [46]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation